In [1]:
import numpy as np
import matplotlib.pyplot as plt
import librosa as lb
import pickle
import glob as glob

In [2]:
def load_audio(filename, Fs = 16000):
    
    x,sr=lb.load(filename,sr=Fs)
    
    return x

In [3]:
import glob as glob
def readDir(dirname, Fs = 16000):
    
    x=[]
    samples_10s=Fs*10
    files=glob.glob(dirname+'\*.wav')
    for i in range(len(files)):
        a=load_audio(files[i])
        No_sample=a.size
        
        if No_sample%samples_10s==0:
            a=a
        else:
            t=int(No_sample/samples_10s)
            no_zerostopad=samples_10s*(t+1)-No_sample
            a=np.append(a,[np.zeros(no_zerostopad)])
        
        frames=len(a)/samples_10s
        a=np.array_split(a,frames)
        x.extend(a)
    x=np.array(x)
        
    return x

In [4]:
def audio2mfcc(x,Fs = 16000):
    n=20
    X=[]
    for i in range(len(x)):
        a=lb.feature.mfcc(x[i],n_mfcc=n,sr=Fs)
        X.append(a)
    X=np.array(X)

    return X 

In [6]:
def framewise(x,y):#tagging framewise  of training data
    
    music=np.array([1,0])
    speech=np.array([0,1])
    X=x[0]
    
    if np.array_equal(y[:,0],music):
        Y=np.transpose([music]*x[0].shape[1])
        
    elif np.array_equal(y[:,0],speech):
        Y=np.transpose([speech]*x[0].shape[1])
        
        
    for i in range(1,len(x)):
        X=np.concatenate((X,x[i]),axis=1)
        if np.array_equal(y[:,i],music):
            a=np.transpose([music]*x[i].shape[1])
            Y=np.column_stack((Y,a))
        elif np.array_equal(y[:,i],speech):
            a=np.transpose([speech]*x[i].shape[1])
            Y=np.column_stack((Y,a))
            
            
    return X,Y
 

In [186]:
def ros(dirname,i=0,Fs=16000):
    n=20
    
    filename=glob.glob(dirname+'\*.wav')
    filename=sorted(filename)
    x,sr=lb.load(filename[i],sr=Fs)
    x=lb.feature.mfcc(x,n_mfcc=n,sr=Fs)
    print(filename[i])
    return x,filename[i]


In [22]:
def predict(X):#predicting framewise of test data
   
    music=np.array([1,0])
    speech=np.array([0,1])
    
    index=X[:,0].argmax()
    if index==0:
        Y=music
    elif index==1:
        Y=speech
    
    for i in range(1,X.shape[1]):
        index=X[:,i].argmax()
        if index==0:
            Y=np.column_stack((Y,music))
        elif index==1:
            Y=np.column_stack((Y,speech))
    Y_mean=np.mean(Y.T,axis=0)
            
    return Y,Y_mean

In [123]:
def finaltag(P):#final tagging to the test data [<music<0/1>,speech<0/1>]
    max_index=P.argmax()
    min_index=1-max_index
    percent=(P[min_index]/P[max_index])*100
    z=np.zeros((2,1),dtype=int)
    if percent<=20:
        z[max_index]=1
    if percent>=20:
        z[max_index]=1
        z[min_index]=1
    
    return z
    

In [177]:
def audioevent(A,B):#finding the start index  and end index of the continuous frame of music/speech
    music=np.array([1,0])
    speech=np.array([0,1])
    max_index=B.argmax()
    if max_index==0:
        U=music.dot(A)
    if max_index==1:
        U=speech.dot(A)
    cont_zeros = 0
    flag=0
    start=0
    stop=0
    H=np.zeros((1,2))
    n=len(U)
    for i in range(len(U)):
        if (U[i] == 1):
            if flag==0:
                start=i
                flag=1
            if cont_zeros != 0:
                cont_zeros = 0
            
        elif flag:
            cont_zeros+=1
            if cont_zeros > 2:
                stop = i-3;
                a=np.array([start,stop]).reshape(1,2)
                H=np.append(H,a,axis=0)
                flag=0
                count_zeros=0
    if flag:
        stop=n-1
        a=np.array([start,stop]).reshape(1,2)
        H=np.append(H,a,axis=0)
    return H

In [119]:
def time(A):#calculating time of the maximum length  train of similar tag music/speech 
    D=np.diff(A)
    l=D.argmax()
    E=A.T[:,l]
    C=np.add(E,1)
    F=np.multiply(C,0.030)
    return F   

In [11]:
from tensorflow.keras.models import Sequential as sq
from tensorflow.keras.layers import Dense as ds, SimpleRNN as rn
from tensorflow.keras import layers

In [16]:
model=sq()
model.add(ds(20,input_dim =len(X_trainframe[:,0]),activation='relu'))
model.add(ds(10,activation='relu'))
model.add(ds(5,activation='relu'))
model.add(ds(2,activation='sigmoid'))

In [17]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 12        
Total params: 697
Trainable params: 697
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
model.fit(x=X_trainframe.T,y=Y_trainframe.T,epochs=100,verbose=1)

Epoch 1/100
2827/2827 [==============================] - 2s 873us/step - loss: 0.2429 - accuracy: 0.9147
Epoch 2/100
2827/2827 [==============================] - 2s 842us/step - loss: 0.1065 - accuracy: 0.9609
Epoch 3/100
2827/2827 [==============================] - 2s 837us/step - loss: 0.0880 - accuracy: 0.9669
Epoch 4/100
2827/2827 [==============================] - 2s 848us/step - loss: 0.0757 - accuracy: 0.9711
Epoch 5/100
2827/2827 [==============================] - 2s 881us/step - loss: 0.0684 - accuracy: 0.9743
Epoch 6/100
2827/2827 [==============================] - 3s 961us/step - loss: 0.0627 - accuracy: 0.9759
Epoch 7/100
2827/2827 [==============================] - 2s 843us/step - loss: 0.0588 - accuracy: 0.9773
Epoch 8/100
2827/2827 [==============================] - 2s 844us/step - loss: 0.0555 - accuracy: 0.9789
Epoch 9/100
2827/2827 [==============================] - 2s 864us/step - loss: 0.0532 - accuracy: 0.9797
Epoch 10/100
2827/2827 [==============================]

2827/2827 [==============================] - 2s 822us/step - loss: 0.0310 - accuracy: 0.9877
Epoch 79/100
2827/2827 [==============================] - 2s 811us/step - loss: 0.0311 - accuracy: 0.9877
Epoch 80/100
2827/2827 [==============================] - 2s 822us/step - loss: 0.0310 - accuracy: 0.9879
Epoch 81/100
2827/2827 [==============================] - 3s 947us/step - loss: 0.0306 - accuracy: 0.9879
Epoch 82/100
2827/2827 [==============================] - 2s 862us/step - loss: 0.0310 - accuracy: 0.9878
Epoch 83/100
2827/2827 [==============================] - 2s 828us/step - loss: 0.0307 - accuracy: 0.9880
Epoch 84/100
2827/2827 [==============================] - 2s 820us/step - loss: 0.0306 - accuracy: 0.9879
Epoch 85/100
2827/2827 [==============================] - 2s 811us/step - loss: 0.0306 - accuracy: 0.9880
Epoch 86/100
2827/2827 [==============================] - 2s 835us/step - loss: 0.0305 - accuracy: 0.9879
Epoch 87/100
2827/2827 [==============================] - 2

# Training model  using training dataset

In [13]:
x_music = readDir('music_wavs') #music data read
x_speech = readDir('speech_wavs') #speech data read
X = np.concatenate((x_music,x_speech)) #concatinating for train dataset
y_music = np.array([[1,0]]*len(x_music)) #labelling music data
y_speech = np.array([[0,1]]*len(x_speech))#labelling speech data
Y_train = np.transpose(np.concatenate((y_music, y_speech)))#concatinating forlabelling crosponding to train dataset
X_train=audio2mfcc(X)#extracting mfcc features of the train data
X_trainframe,Y_trainframe=framewise(X_train,Y_train)#aggregating framewise data X,Y(mfcc_coeff,frames)
print(X_trainframe.shape,Y_trainframe.shape)

(196, 160000)
(160000,)
196
(289, 20, 313)
(20, 90457) (2, 90457)


In [205]:
def name(D):
    index=D.argmax()
    if index==0:
        return 'music'
    if index==1:
        return 'speech'

# Testing over audio.wav file

In [228]:
X_test,filename=ros('test',0)#laoding test data and extarcting mfcc features dim(20,number of frames) ,i=filename index I/P
prediction=model.predict(X_test.T) #predicting frame wise with dim(2,frames)
R=np.mean(prediction.T,axis=1)# mean of prediction framewise
P_framewise,P_meanframe=predict(prediction.T) #labelling framewise and it labelleb mean along frame
Event_span=audioevent(P_framewise,P_meanframe)# start and end  continuous frame of music/speech 
time_span=time(Event_span)#time span of max continuous length of train of music/speech
Tagged_audio=finaltag(P_meanframe)#final tagging to the audio [music,speech].T
time_spanevent=name(P_meanframe)# timespan of event name
print(R)
print(P_meanframe)
time_spanevent=name(P_meanframe)
print(time_spanevent)
print(time_span)
print(Tagged_audio.T)

test\S001.wav
[0.10699449 0.89300567]
[0.10543131 0.89456869]
speech
[0.03 5.16]
[[0 1]]


In [227]:
#overview of result
Task1=[filename,time_spanevent,time_span]
Task2=[Tagged_audio.T]
print(Task1)
print(Task2)

['test\\S001.wav', 'speech', array([0.03, 5.16])]
[array([[0, 1]])]
